In [10]:
from openai import OpenAI
TOGETHER_AI_API_KEY = 'f41b1428204c49e0c8ae6f7b020557297806c591b1800d7cb2d241ab5b065805'
def generate_diffuser_text(thread_info: str, toxic_comments: list) -> str:
    """
    Generate a response aimed at diffusing the toxicity identified in the thread,
    incorporating the context of the thread and the toxic comments.

    :param thread_info: A string containing the compiled content of the thread.
    :param toxic_comments: A list of strings, each a comment identified as toxic.
    :return: A string containing the AI-generated diffuser text.
    """
    client = OpenAI(api_key="YOUR_API_KEY", base_url="https://api.together.xyz")
    toxic_summary = " ".join(toxic_comments)  # Combine toxic comments into a single string for simplicity
    diffuser_prompt = f"With the context of this thread: {thread_info} and the following toxic comments: {toxic_summary}, generate a response that can help diffuse the situation."
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI trained to promote positive interactions and mitigate toxicity. Here's the context and toxic comments you need to address."},
            {"role": "user", "content": diffuser_prompt}
        ],
        model="google/gemma-7b-it",
    )
    diffuser_text = chat_completion.choices[0].message.content
    return diffuser_text

In [3]:
import os

from atproto import (
    CAR,
    AtUri,
    Client,
    FirehoseSubscribeReposClient,
    firehose_models,
    models,
    parse_subscribe_repos_message,
)
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Bluesky credentials
BLUESKY_USERNAME = "paritoshk.bsky.social"
BLUESKY_PASSWORD = "Pari2798!"

# Create a Bluesky client
client = Client("https://bsky.social")
firehose = FirehoseSubscribeReposClient()

In [7]:
def process_operation(
    op: models.ComAtprotoSyncSubscribeRepos.RepoOp,
    car: CAR,
    commit: models.ComAtprotoSyncSubscribeRepos.Commit,
):
    uri = AtUri.from_str(f"at://{commit.repo}/{op.path}")

    if op.action == "create":
        if not op.cid:
            return

        record = car.blocks.get(op.cid)
        if not record:
            return

        record = {
            "uri": str(uri),
            "cid": str(op.cid),
            "author": commit.repo,
            **record,
        }
    

        if uri.collection == models.ids.AppBskyFeedPost:
            # This logs the text of every post off the firehose.
            # Just for fun :)
            # Delete before actually using
            print(record['text'])
        
            if "stop-tox" in record["text"]:
                # get some info about the poster, their posts, and the thread they tagged the bot in
                poster_posts = client.get_author_feed(
                    actor=record["author"], cursor=None, filter=None, limit=100
                ).feed
                poster_follows = client.get_follows(actor=record["author"]).follows
                poster_profile = client.get_profile(actor=record["author"])
                posts_in_thread = client.get_post_thread(uri=record["uri"])

                # send a reply to the post
                record_ref = {"uri": record["uri"], "cid": record["cid"]}
                reply_ref = models.AppBskyFeedPost.ReplyRef(
                    parent=record_ref, root=record_ref
                )
                client.send_post(
                    reply_to=reply_ref,
                    text=f"Hey, {poster_profile.display_name}. You have {len(poster_posts)} posts and {len(poster_follows)} follows. Your bio is: {poster_profile.description}. There are {len(posts_in_thread)} posts in the thread.",
                )

        # elif uri.collection == models.ids.AppBskyFeedLike:
        #     print("Created like: ", record)
        # elif uri.collection == models.ids.AppBskyFeedRepost:
        #     print("Created repost: ", record)
        # elif uri.collection == models.ids.AppBskyGraphFollow:
        #     print("Created follow: ", record)

    if op.action == "delete":
        # Process delete(s)
        return

    if op.action == "update":
        # Process update(s)
        return

    return posts_in_thread

In [6]:
def on_message_handler(message: firehose_models.MessageFrame) -> None:
    commit = parse_subscribe_repos_message(message)
    if not isinstance(
        commit, models.ComAtprotoSyncSubscribeRepos.Commit
    ) or not isinstance(commit.blocks, bytes):
        return
    car = CAR.from_bytes(commit.blocks)
    for op in commit.ops:
        process_operation(op, car, commit)


def main() -> None:
    client.login(BLUESKY_USERNAME, BLUESKY_PASSWORD)
    print("🤖 Bot is listening")
    firehose.start(on_message_handler)


In [8]:
client = OpenAI(api_key=TOGETHER_AI_API_KEY, base_url="https://api.together.xyz")

In [11]:
chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI trained to mitigate toxic conversations and promote positive interactions. Here's the context and what you need to address: " + "Nina: Fuck you, Tosh: Sorry please, Nina: Nice goodbye"},
            {"role": "user", "content": "stoptox now"}
        ],
        model="google/gemma-7b-it",  # Make sure to replace with the correct model identifier if needed
    )


InternalServerError: Error code: 500 - {'error': {'message': '(unknown path)\n  Error: Unable to call `raise_exception`, which is undefined or falsey', 'type': 'server_error', 'param': None, 'code': None}}

In [ ]:
# Mock function to simulate replicate.run behavior
import replicate 
REPLICATE_API_TOKEN = 'r8_7ICSWEeEQdH6SyLPpcjU1723lKbVExK1kqyIx'
def mock_replicate_run(model, input):
    # Simulated response from the AI model
    return [{"text": "It's important to remember the impact our words can have. Let's strive for understanding and kindness."}]

# Overwrite the replicate.run function with the mock function for testing
replicate.run = mock_replicate_run

# Example posts in the thread
posts_in_thread = [
    {"text": "I really enjoyed the event, it was very well organized."},
    {"text": "Absolutely, kudos to everyone involved."},
    {"text": "I didn't like it. Felt it was a waste of time."},
]

# Example toxic message
toxic_message = "I didn't like it. Felt it was a waste of time."

# Test the function
response = send_prompt_to_ai(posts_in_thread, toxic_message)

print("AI-generated response to mitigate toxicity:")
print(response)
